# The Full Script

## 1. Loading libraries

In [1]:
# Import necessary packages
import pyautogui as auto
import pygetwindow as get
import requests
import time
import clipboard
import re
import pandas as pd
import numpy as np

# Word libraries
from docx import Document

# Web libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Remote
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

# System libraries
from datetime import datetime

# System libraries
import os, sys
import subprocess

## 2. Functions

In [2]:
# Clicking buttons
def click_button_by_xpath(driver, xpath, timeout=10):
    """
    Waits for a button to be clickable using its XPath and clicks it.

    Args:
        driver: Selenium WebDriver instance.
        xpath (str): XPath string to locate the button.
        timeout (int): Max wait time in seconds.

    Returns:
        True if clicked successfully, False otherwise.
    """
    try:
        wait = WebDriverWait(driver, timeout)
        button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        button.click()
        return True
    except Exception as e:
        print(f"Failed to click button: {e}")
        return False

# Date conversion
def format_date_part(date_str):
    """Convert 'YYYY.MM.DD' to 'dd MMM YYYY', fixing common formatting errors"""
    try:
        clean_str = date_str.strip().rstrip('.')  # remove trailing dot
        return datetime.strptime(clean_str, "%Y.%m.%d").strftime("%d %b %Y")
    except ValueError:
        return date_str.strip()  # return cleaned original if still unparseable

# Reformat date entries
def format_date_column(date_entry):
    """
    Handle full, range, and partial date formats:
    - '2025.04.02 ~ 2025.04.06' → '02 Apr 2025 ~ 06 Apr 2025'
    - '2025.04.02~' → '02 Apr 2025~'
    - '2025.04.02' → '02 Apr 2025'
    """
    # Split on ~ (tilde) with optional whitespace
    parts = re.split(r'\s*~\s*', date_entry)
    formatted_parts = [format_date_part(p) for p in parts]
    return ' ~ '.join(formatted_parts)

# Enter Drug Name
def fill_input_field_by_id(driver, field_id, value, timeout=10):
    """
    Waits for an input field by ID, clears it, enters the given value,
    and asserts that the value was set correctly.

    Args:
        driver: Selenium WebDriver instance.
        field_id (str): The ID of the input field.
        value (str): The value to enter.
        timeout (int): Time to wait for the input field to be visible.

    Returns:
        True if successful, False otherwise.
    """
    try:
        wait = WebDriverWait(driver, timeout)
        input_box = wait.until(EC.visibility_of_element_located((By.ID, field_id)))

        input_box.clear()
        input_box.send_keys(value)

        # Verify the value
        return input_box.get_attribute("value") == value
    except Exception as e:
        print(f"Error filling input field '{field_id}': {e}")
        return False

# Split date range data
def split_date_range(date_range_str, delimiter="~", date_format="%d %b %Y"):
    """
    Splits a date range string into two datetime.date objects.

    Args:
        date_range_str (str): The input string (e.g. '02 Apr 2025 ~ 06 Apr 2025').
        delimiter (str): The character that separates the two dates.
        date_format (str): The format of the input dates.

    Returns:
        tuple: (start_date, end_date) as datetime.date objects.
    """
    try:
        parts = [d.strip() for d in date_range_str.split(delimiter)]
        start_date = datetime.strptime(parts[0], date_format).date()
        start_date_str = start_date.strftime("%d %b %Y")
        # Handle optional end date
        end_date = None
        end_date_str = None
        if len(parts) > 1 and parts[1]:  # not empty string
            end_date = datetime.strptime(parts[1], date_format).date()
            end_date_str = end_date.strftime("%d %b %Y")
        return start_date_str, end_date_str
    except ValueError as e:
        raise ValueError(f"Could not parse date range: {e}")

# Enter Reason for administration
def click_radio_by_label(driver, label_text):
    """
    Clicks a radio button whose associated label has the specified text.
    Returns True if successful, False if label not found.
    """
    try:
        wait = WebDriverWait(driver, 10)
        labels = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'label[data-bind*="ChoiceText"]')
        ))

        for label in labels:
            if label.text.strip() == label_text:
                parent = label.find_element(By.XPATH, '..')
                radio_div = parent.find_element(By.CSS_SELECTOR, '.radioArea.radio-btn')
                radio_div.click()
                return True
        return False  # Label not found
    except Exception as e:
        print(f"Error clicking radio button: {e}")
        return False

# Debug clicking
def debug_click(driver):
    # Take a screenshot before clicking
    driver.save_screenshot("debug_select2.png")
    # Log current DOM elements
    print(driver.page_source)
    # Check if JS errors are preventing dropdown render
    for entry in driver.get_log('browser'):
        print(entry)


# Select from dropdown menu "Specify adverse event(s)"
def select_select2_option(driver, container_selector, option_text, timeout=10):
    """
    Selects an option from a Select2 dropdown by visible text.

    Args:
        driver: Selenium WebDriver instance.
        container_selector (tuple): Locator for the clickable Select2 container (usually a <span> or similar).
        option_text (str): The visible text to select.
        timeout (int): Timeout in seconds for waits.

    Returns:
        True if successful, False otherwise.
    """
    try:
        wait = WebDriverWait(driver, timeout)

        # Click the container to activate the Select2 dropdown
        select2_container = wait.until(EC.element_to_be_clickable(container_selector))
        select2_container.click()
        
        # Wait for the search input to be visible and enter the desired text
        search_input = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "select2-search__field")))
        search_input.clear()
        search_input.send_keys(option_text)

        # Wait for the dropdown result and click the one matching the option text
        dropdown_option_xpath = f"//li[contains(@class, 'select2-results__option') and contains(text(), '{option_text}')]"
        option_to_select = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_option_xpath)))
        option_to_select.click()

        print(f"[✓] Selected '{option_text}' from dropdown.")
        return True
        
    except Exception as e:
        print(f"Error selecting Select2 option '{option_text}': {e}")
        return False

# Select from "Medical history"
def select_select2_keyword_option(driver, keyword, timeout=10):
    try:
        wait = WebDriverWait(driver, timeout)

        # Step 1: Click the Select2 container to open the dropdown
        container = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, ".select2-selection--multiple")))
        container.click()

        # Step 2: Type keyword into the search field
        search_input = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "input.select2-search__field")))
        search_input.clear()
        search_input.send_keys(keyword)

        # Step 3: Wait for result list to populate
        result_item = wait.until(EC.element_to_be_clickable((
            By.XPATH, f"//li[contains(@class, 'select2-results__option') and contains(text(), '{keyword}')]"
        )))

        # Step 4: Click the result item
        result_item.click()

        print(f"[✓] Selected an option containing '{keyword}'")
        return True

    except Exception as e:
        print(f"[✗] Failed to select option containing '{keyword}': {e}")
        return False

# Select from dropdown menu "Unit"
def select_chosen_option(driver, dropdown_id, option_text, timeout=10):
    """
    Selects an option from a Chosen.js dropdown by visible text.

    Args:
        driver: Selenium WebDriver instance
        dropdown_id (str): The ID of the original <select> element (not the _chosen div)
        option_text (str): The visible text of the option to select
        timeout (int): Maximum wait time in seconds

    Returns:
        True if selection was successful, False otherwise
    """
    try:
        wait = WebDriverWait(driver, timeout)

        # Build Chosen container ID
        chosen_container_id = f"{dropdown_id}_chosen"

        # Click the Chosen dropdown to open it
        toggle = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#{chosen_container_id} .chosen-single")))
        toggle.click()

        # Wait for and click the desired option
        option_xpath = f"//div[@id='{chosen_container_id}']//li[contains(@class, 'active-result') and normalize-space(text())='{option_text}']"
        option = wait.until(EC.element_to_be_clickable((By.XPATH, option_xpath)))
        option.click()

        return True
    except Exception as e:
        print(f"Error selecting Chosen.js option '{option_text}': {e}")
        return False

# Separate number and unit
def split_number_and_unit(dose):
    """
    Splits a string like '5mg' into ('5', 'mg').

    Args:
        dose (str): The input string containing number and unit.

    Returns:
        tuple: (number_part, text_part) or (None, None) if no match.
    """
    match = re.match(r"(\d+)([a-zA-Z]+)", dose)
    if match:
        number_part, text_part = match.groups()
        return number_part, text_part
    return None, None

# Select date from datepicker
def select_date_from_datepicker(driver, calendar_button_selector, target_date_str, container_id=None, timeout=10):
    """
    Opens a datepicker within a specific container and selects a given date.

    Args:
        driver: Selenium WebDriver instance.
        calendar_button_selector (str): CSS selector for the calendar open button.
        target_date_str (str): Date in 'dd MMM yyyy' format, e.g., '15 Apr 2025'.
        container_id (str): ID of the parent container <div> that holds the datepicker (optional).
        timeout (int): Max wait time for each action.

    Returns:
        True if successful, False otherwise.
    """
    from datetime import datetime
    wait = WebDriverWait(driver, timeout)
    target_date = datetime.strptime(target_date_str, "%d %b %Y")
    target_day = str(target_date.day)
    target_month = target_date.strftime("%B")  # Full month name
    target_year = str(target_date.year)

    try:
        # 1. Click the calendar button to open the datepicker
        calendar_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, calendar_button_selector)))
        calendar_button.click()

        # Create scoped XPath prefix if container_id is provided
        scope_prefix = f"//div[@id='{container_id}']" if container_id else ""

        # 2. Click the switch button to go to month/year view
        switch_xpath = f"{scope_prefix}//div[contains(@class, 'datepicker-days')]//th[contains(@class, 'switch')]"
        switch_button = wait.until(EC.element_to_be_clickable((By.XPATH, switch_xpath)))
        switch_button.click()
        time.sleep(0.2)

        # 3. Click again to go to year view
        year_switch_xpath = f"{scope_prefix}//div[contains(@class, 'datepicker-months')]//th[contains(@class, 'switch')]"
        year_switch = wait.until(EC.element_to_be_clickable((By.XPATH, year_switch_xpath)))
        year_switch.click()
        time.sleep(0.2)

        # 4. Select the year
        year_xpath = f"{scope_prefix}//span[contains(@class, 'year') and text()='{target_year}']"
        year_elem = wait.until(EC.element_to_be_clickable((By.XPATH, year_xpath)))
        year_elem.click()
        time.sleep(0.2)

        # 5. Select the month
        month_xpath = f"{scope_prefix}//span[contains(@class, 'month') and text()='{target_date.strftime('%b')}']"
        month_elem = wait.until(EC.element_to_be_clickable((By.XPATH, month_xpath)))
        month_elem.click()
        time.sleep(0.2)

        # 6. Select the day
        day_xpath = f"{scope_prefix}//td[contains(@class, 'day') and text()='{target_day}']"
        day_elem = wait.until(EC.element_to_be_clickable((By.XPATH, day_xpath)))
        day_elem.click()

        return True
    except Exception as e:
        print(f"Failed to select date {target_date_str}: {e}")
        return False


## 3. Data Processing

In [3]:
# 1. 데이터 가져오기
# Load the .docx file
doc = Document("test_doc.docx")  # 파일 이름 입력

# Extract text lines (paragraphs)
lines = [para.text.strip() for para in doc.paragraphs if para.text.strip()]

# 2. 데이터 처리
# Process the data
cleaned_lines = []

for line in lines:
    if line.startswith("·"):
        line = line[1:].strip()  # Remove bullet
    parts = line.split(':', 1)
    drug_name = parts[0].strip()
    rest = parts[1].strip() if len(parts) > 1 else ''
    cleaned_line = f"{drug_name}/ {rest}"
    # Split by '/' and strip spaces
    fields = [field.strip() for field in cleaned_line.split('/')]
    cleaned_lines.append(fields)

# Optionally inspect the longest row to ensure consistent column count
max_len = max(len(row) for row in cleaned_lines)
for row in cleaned_lines:
    while len(row) < max_len:
        row.append("")  # pad with empty strings if any row is shorter

# 3. 데이터 테이블로 정리
# Create DataFrame
columns = [f"Column_{i+1}" for i in range(max_len)]
df = pd.DataFrame(cleaned_lines, columns=columns)

# Set column names
df.columns = [
    "Drug Name", "Date", "Reason for Administration", "Dose", "Frequency", "Route"
]

# Apply the date conversion function to the "Date" column
df["Date"] = df["Date"].apply(format_date_column)

# Display the DataFrame
df

,Drug Name,Date,Reason for Administration,Dose,Frequency,Route
0,Vacrax oint.,02 Apr 2025 ~ 06 Apr 2025,cold sores,1ea,PRN,topical
1,K-cab.,02 Apr 2025 ~,prophylaxis for GI trouble,50mg,PRN,PO
2,ondansetron inj,02 Apr 2025 ~,premedication before CTx,4mg,PRN,IV
3,Pegfilgrastim,11 Apr 2025 ~,prophylaxis for neutropenia,0.6ml,PRN,SC
4,Grasin inj.,14 Apr 2025 ~ 14 Apr 2025,Neutropenic fever,300mcg,qd,SC
5,SmofKabiven inj.,14 Apr 2025 ~ 24 Apr 2025,nutrition support,986ml,qd,IV
6,Pheniramine inj.,15 Apr 2025 ~ 20 Apr 2025,pre med for transfusion,4mg,PRN,IV
7,Magnesium Sulfate inj. 10%,15 Apr 2025 ~ 24 Apr 2025,nutrition support,4g,qd,IV
8,Humalog inj,14 Apr 2025 ~ 24 Apr 2025,DM,20IU,QD,IV
9,KCl-20 inj.,15 Apr 2025 ~ 20 Apr 2025,nutrition support,40mEq,qd,IV


## 4. Open Viedoc

In [4]:
# 1. Viedoc 열기
# Initialize the viedoc instance
path = chromedriver_autoinstaller.install()
viedoc=webdriver.Chrome()
viedoc.get('https://v4.viedoc.net')

# 2. Concomitant Medication 화면 띄우기 (수동으로 할 것)

## 5. Add Viedoc Concomitant Medication item(Must proceed only when Concomitant Medication window has been opened)

In [15]:
# 모든 리스트 아이템에 대해 반복
for index, row in df.iterrows():
    print(index, row)
    # 약물 속성 지정
    drug_name = row["Drug Name"]
    roa = row["Reason for Administration"]  
    dose = row["Dose"]
    freq = row["Frequency"]
    route = row["Route"]
    date_range = row["Date"]
    # Medical history 필터(아직 적용 불가이므로 걸러냄)
    md = ["Dyslipidemia", "diabetes mellitus" "Hypertension", "Herniated nucleus pulposis", "poor oral intake", "Increased liver enzymes", "Postmenopausal"]
    if roa not in md :
        # 0. Add New 버튼 클릭하기
        # Wait for the parent element to be present
        wait = WebDriverWait(viedoc, 15)
        
        parent_element = wait.until(
            EC.presence_of_element_located((By.ID, "events-COMMON_CM"))
        )
        
        # Click on "Add New" button
        xpath_add_common = '//*[@id="events-COMMON_CM"]//a[@class="fb" and @title="Add new" and contains(@href, "FormData/AddCommon")]'
        success = click_button_by_xpath(viedoc, xpath_add_common)
        
        if success:
            print("Add new button clicked.")
        else:
            print("Failed to click the button.")
        # 1. 약물 이름 입력
        if fill_input_field_by_id(viedoc, "CMTRT", drug_name):
            print("Input filled successfully.")
        else:
            print("Failed to fill input.")

        # 2. Reason for Administration 입력
        # Set to "Adverse event" if it belongs to any of the following
        if roa == "Neutropenic fever":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#1 - Description: Neutropenic fever - Start date: 14 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "cold sores":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#2 - Description: cold sores  - Start date: 02 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "thrombocytopenia":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#3 - Description: thrombocytopenia - Start date: 10 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "thrombocytopenia":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#4 - Description: thrombocytopenia - Start date: 15 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "steroid induced hyperglycemia":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#5 - Description: steroid induced hyperglycemia  - Start date: 03 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "oral mucositis":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#6 - Description: oral mucositis  - Start date: 14 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "diarrhea":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#7 - Description: diarrhea  - Start date: 16 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "thrombocytopenia":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#8 - Description: thrombocytopenia  - Start date: 21 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "anemia":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#9 - Description: anemia  - Start date: 15 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "petechiae, both buttock":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#10 - Description: antibiotic-induced petechiae, both buttock  - Start date: 16 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "Herpes labialis":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#11 - Description: Herpes labialis - Start date: 20 Apr 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        if roa == "seroma, Lt axilla":
            # Loop through labels to find the one with text "Adverse event"
            if click_radio_by_label(viedoc, "Adverse event"):
                print("Radio button clicked successfully.")
                # Usage: click the visible Select2 <span> that opens the dropdown
                selector = "#12 - Description: seroma, Lt axilla  - Start date: 02 May 2025"
                if select_select2_option(viedoc, (By.CSS_SELECTOR, ".select2-container"), selector):
                    print("Option selected successfully.")
                else:
                    print("Failed to select the option.")
            else:
                print("Radio button with label 'Adverse event' not found.")

        # Set to "Supportive care treatment(s)" if it belongs to any of below
        if roa == "nutrition support":
            # Loop through labels to find the one with text "Supportive care treatment(s)"
            if click_radio_by_label(viedoc, "Supportive care treatment(s)"):
                print("Radio button clicked successfully.")
                # Enter the data into the field
                if fill_input_field_by_id(viedoc, "CMINDCSUPTRT", roa):
                    print("Specify entered successfully.")
                else:
                    print("Failed to enter Specify.")
            else:
                print("Radio button with label 'Supportive care treatment(s)' not found.")

        # Set to "Other" if it belongs to any of below
        if "prophylaxis" in roa:
            # Loop through labels to find the one with text "Other"
            if click_radio_by_label(viedoc, "Other"):
                print("Radio button clicked successfully.")
                # Enter the data into the field
                if fill_input_field_by_id(viedoc, "CMINDCOTH", roa):
                    print("Specify entered successfully.")
                else:
                    print("Failed to enter Specify.")
            else:
                print("Radio button with label 'Other' not found.")

        if "pre med" in roa or "premed" in roa:
            # Loop through labels to find the one with text "Other"
            if click_radio_by_label(viedoc, "Other"):
                print("Radio button clicked successfully.")
                # Enter the data into the field
                if fill_input_field_by_id(viedoc, "CMINDCOTH", roa):
                    print("Specify entered successfully.")
                else:
                    print("Failed to enter Specify.")
            else:
                print("Radio button with label 'Other' not found.")

        # 3. Enter Dose and Unit
        # 숫자와 단위 분리
        num, unit = split_number_and_unit(dose)
        if num and unit:
            print("Number:", num)
            print("Unit:", unit)
        else:
            # This is "Not Applicable" case
            num = dose
            print("Number:", num)
            print("This is unit less")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDSTXT", num):
                print("Dose entered successfully.")
            else:
                print("Failed to enter Dose.")
            if select_chosen_option(viedoc, "CMDOSU", "Not applicable"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
                
        # Enter the data into the field
        if fill_input_field_by_id(viedoc, "CMDSTXT", num):
            print("Dose entered successfully.")
        else:
            print("Failed to enter Dose.")
        
        if unit == "mg":
            if select_chosen_option(viedoc, "CMDOSU", "Milligram"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "g":
            if select_chosen_option(viedoc, "CMDOSU", "Gram"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "ml":
            if select_chosen_option(viedoc, "CMDOSU", "Millilitre"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "mcg":
            if select_chosen_option(viedoc, "CMDOSU", "Microgram"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "IU":
            if select_chosen_option(viedoc, "CMDOSU", "International Unit"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "IU":
            if select_chosen_option(viedoc, "CMDOSU", "International unit"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "ea":
            if select_chosen_option(viedoc, "CMDOSU", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if unit == "V":
            if select_chosen_option(viedoc, "CMDOSU", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        if "Eq" in unit:
            if select_chosen_option(viedoc, "CMDOSU", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSUOTH", unit):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
                
        # 4. Dose Form 입력
        # Dose name classification
        if "oint" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Ointment"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if "suspension" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Suspension"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if "tab" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Tablet"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")   
        
        if "cap" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Capsule"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")  
        
        if "lotion" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Cream"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")  
        
        if "inj" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")   
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSFRMOTH", "Injection"):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.")
        
        # Drug specific classification
        if "K-cab" in drug_name:
            if select_chosen_option(viedoc, "CMDOSFRM", "Tablet"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")    
        
        if drug_name == "Pegfilgrastim":
            if select_chosen_option(viedoc, "CMDOSFRM", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")   
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMDOSFRMOTH", "Injection"):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.") 
        
        if drug_name == "Solondo":
            if select_chosen_option(viedoc, "CMDOSFRM", "Powder"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.") 

        # 5. Enter Frequency
        freq = df.loc[1, "Frequency"]
        if freq == "PRN":
            if select_chosen_option(viedoc, "CMDOSFRQ", "As needed"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if freq == "qd":
            if select_chosen_option(viedoc, "CMDOSFRQ", "Once daily"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if freq == "bid":
            if select_chosen_option(viedoc, "CMDOSFRQ", "Twice daily"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if freq == "tid":
            if select_chosen_option(viedoc, "CMDOSFRQ", "Three times a day"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")

        # 6. Enter Route
        if route == "topical":
            if select_chosen_option(viedoc, "CMROUTE", "Topical"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if route == "PO":
            if select_chosen_option(viedoc, "CMROUTE", "Oral"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if route == "SC":
            if select_chosen_option(viedoc, "CMROUTE", "Subcutaneous"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")
        
        if route == "IV":
            if select_chosen_option(viedoc, "CMROUTE", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")   
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMROUTEOTH", route):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.") 
        
        if route == "gargle":
            if select_chosen_option(viedoc, "CMROUTE", "Other"):
                print("Chosen option selected successfully.")
            else:
                print("Failed to select Chosen option.")   
            # Enter the data into the field
            if fill_input_field_by_id(viedoc, "CMROUTEOTH", route):
                print("Unit entered successfully.")
            else:
                print("Failed to enter Unit.") 

        else:
            print("아직 등록되지 않은 약물입니다. 프로그램 코드에 조건을 추가해주세요.")

        # 7. Enter Start date      
        # Split on "~" and strip any surrounding whitespace
        start_date, end_date = split_date_range(date_range)
        print("Start:", start_date)
        print("End:", end_date)
        
        # Select date from datepicker
        container_id = "CMSTDAT"
        calendar_button_selector = "#" + container_id + ' a[data-dtp-action="datepicker"]'
        if select_date_from_datepicker(viedoc, calendar_button_selector, start_date, container_id):
            print("Date selected successfully.")
        else:
            print("Date selection failed.")

        # 8. Enter Ongoing? and End date(if not ongoing)
        if end_date == None:
            # Loop through labels to find the one with text "Yes"
            if click_radio_by_label(viedoc, "Yes"):
                print("Radio button clicked successfully.")
            else:
                print("Radio button with label 'Yes' not found.")
        if end_date != None:
            # Loop through labels to find the one with text "No"
            if click_radio_by_label(viedoc, "No"):
                print("Radio button clicked successfully.")
            else:
                print("Radio button with label 'No' not found.") 
            # Select date from datepicker
            container_id = "CMENDAT"
            calendar_button_selector = "#" + container_id + ' a[data-dtp-action="datepicker"]'
            if select_date_from_datepicker(viedoc, calendar_button_selector, end_date, container_id):
                print("Date selected successfully.")
            else:
                print("Date selection failed.")

        else:
            print("아직 등록되지 않은 약물입니다. 프로그램 코드에 조건을 추가해주세요.")
                
        # 9. 저장하기
        xpath_save_changes = '//*[@id="form_link_save"]'
        success = click_button_by_xpath(viedoc, xpath_save_changes)
        if success:
            print("Save changes button clicked.")
        else:
            print("Failed to click the Save changes button.")
    else:
        print("아직 등록되지 않은 약물입니다. 프로그램 코드에 조건을 추가해주세요.")

0 Drug Name                                 Vacrax oint.
Date                         02 Apr 2025 ~ 06 Apr 2025
Reason for Administration                   cold sores
Dose                                               1ea
Frequency                                          PRN
Route                                          topical
Name: 0, dtype: object
Add new button clicked.
Input filled successfully.
Radio button clicked successfully.
[✓] Selected '#2 - Description: cold sores  - Start date: 02 Apr 2025' from dropdown.
Option selected successfully.
Number: 1
Unit: ea
Dose entered successfully.
Chosen option selected successfully.
Unit entered successfully.
Chosen option selected successfully.
Chosen option selected successfully.
Chosen option selected successfully.
Start: 02 Apr 2025
End: 06 Apr 2025
Date selected successfully.
Radio button clicked successfully.
Date selected successfully.
Save changes button clicked.
1 Drug Name                                        K-cab.
Date    

KeyboardInterrupt: 